<a href="https://colab.research.google.com/github/Kzis/my-saprk/blob/main/my_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [23]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [24]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [25]:
!pip install -q findspark

In [26]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [27]:
import findspark
findspark.init()

In [28]:
findspark.find()

'/content/spark-3.0.0-bin-hadoop3.2'

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [30]:
spark